In [1]:
import sys
sys.path.append(r'C:\Users\ICN_admin\Documents\icn\icn_bids')
sys.path.append(r'C:\Users\ICN_admin\Documents\icn')
import icn_bids
from bids import BIDSLayout
import mne_bids
import mne
import numpy as np
import os
import pybv

In [ ]:
def preprocess_mov(mov_dat):
    """
    clean a TTL signal into continous movement block
    """
    # the TIME OFF in the TTL signal is ~50 ms
    MOV_ON = False
    mov_new = np.zeros(mov_dat.shape[0])
    mov_new[0] = mov_dat[0]
    mov_on_set = 0
    for i in range(mov_dat.shape[0]):
        if i > 0 and mov_dat[i] > 1:
            MOV_ON = True
            mov_on_set = i
        if (i - mov_on_set) > 100 and mov_dat[i] < 1:
            mov_new[i] = 0
            MOV_ON = False
        if MOV_ON is True:
            mov_new[i] = 1
    return mov_new

def set_chtypes(vhdr_raw):
    """
    define MNE RawArray channel types
    """
    #print('Setting new channel types...')
    remapping_dict = {}
    for ch_name in vhdr_raw.info['ch_names']:
        if ch_name.startswith('ECOG'):
            remapping_dict[ch_name] = 'ecog'
        elif ch_name.startswith(('LFP', 'STN')):
            remapping_dict[ch_name] = 'seeg'
        elif ch_name.startswith('EMG'):
            remapping_dict[ch_name] = 'emg'
        # mne_bids cannot handle both eeg and ieeg channel types in the same data
        elif ch_name.startswith('EEG'):
            remapping_dict[ch_name] = 'misc'
        elif ch_name.startswith(('MOV', 'ANALOG', 'ROT', 'ACC', 'AUX', 'X', 'Y', 'Z')):
            remapping_dict[ch_name] = 'misc'
        else:
            remapping_dict[ch_name] = 'misc'
    vhdr_raw.set_channel_types(remapping_dict, verbose=False)
    return vhdr_raw

In [ ]:
# now setup as function

# I defined a copy of the BIDS Beijing dataset here
PATH_BEIJING = r'C:\Users\ICN_admin\Documents\Decoding_Toolbox\write_Beijing_again\BIDS_here'

# and want to save the updated files in this directory 
PATH_UPDATE = r'C:\Users\ICN_admin\Documents\Decoding_Toolbox\write_Beijing_again\BIDS_update_1003'
# using pybids I read the run files 
layout = BIDSLayout(PATH_BEIJING)
for run_file in layout.get(extension='.vhdr'):
    print("writing "+str(run_file))
    #run_file = layout.get(extension='.vhdr')[0]

    # get the bids_path
    entities = mne_bids.get_entities_from_fname(run_file)
    bids_path = mne_bids.BIDSPath(subject=entities["subject"], session=entities["session"], task=entities["task"], \
        run=entities["run"], acquisition=entities["acquisition"], datatype="ieeg", root=PATH_BEIJING)

    # then read the brainvision file 
    raw_arr = mne.io.read_raw_brainvision(run_file)

    entities = mne_bids.get_entities_from_fname(run_file)

    # and the electrode.tsv file 
    elec_file = layout.get(extension='.tsv', suffix="electrodes", space="mni", subject=entities["subject"])[0].get_df()

    # first, add the fixed channel to the BIDS Data and create a new MNE RawArray using that 
    data_ = raw_arr.get_data()
    print(data_.shape)

    # then I add for every TTL channel the cleaned version 
    add_ = []
    add_label = []
    for ch_idx, ch in enumerate(raw_arr.ch_names):
        if "TTL" in ch:
            add_.append(preprocess_mov(data_[ch_idx, :]))
            add_label.append("TTL_"+str(len(add_))+"_clean")

    # stack the results to the previous brainvision data 
    data_new = np.concatenate((data_, np.vstack(add_)), axis=0)
    ch_l = raw_arr.ch_names.copy()
    [ch_l.append(l) for l in add_label] # updated channel list is now in ch_l 
    
    folder_dummy = r'C:\Users\ICN_admin\Documents\Decoding_Toolbox\write_Beijing_again\folder_dummy'
    pybv.write_brainvision(data=data_new, \
                       sfreq=raw_arr.info["sfreq"], 
                       ch_names=ch_l,
                      fname_base='dummy', folder_out=folder_dummy)

    # I will now read this file using mne io again 
    raw_arr_read_ = mne.io.read_raw_brainvision(os.path.join(folder_dummy, 'dummy.vhdr'))

    # now I define the new electrode tsv, that will be hopefully written out by mne_bids
    elec = np.empty(shape=(len(raw_arr_read_.ch_names), 3))
    for ch_idx, ch in enumerate(raw_arr_read_.ch_names):
        try:
            elec[ch_idx, :] = elec_file[elec_file["name"] == ch][["x", "y", "z"]]
        except:
            elec[ch_idx, :] = ["NaN", "NaN", "NaN"]

    montage = mne.channels.make_dig_montage(ch_pos=dict(zip(raw_arr_read_.ch_names, elec)), \
                                            coord_frame='mri')

    raw_arr_read_.set_montage(montage, on_missing='warn')

    # now adapt the channel types with Jonathan's function 
    raw_arr_read_ = set_chtypes(raw_arr_read_)

    raw_arr_read_.info['line_freq'] = 50 # otherwise we cannot write BIDS
    
    bids_path_update = bids_path
    bids_path_update.root = PATH_UPDATE

    mne_bids.write_raw_bids(raw=raw_arr_read_, bids_path=bids_path_update, overwrite=True)
    